# Datenanalyse: botanischer Garten
## Gruppenprojekt - Lexer & Maderbök

### Daten laden

In [23]:
import pandas
from pandas import read_csv

data = read_csv(
    "data/iris.csv",
    sep=",",
    decimal=".",
    skiprows=None,
    usecols=range(1, 6)
)

### 1. Explorative Datenanalyse

#### Statische Gesamtauswertung

In [24]:
data.describe()

,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,species
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [25]:
data["sepal_length_cm"].plot()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

### 2. Unsupervised Learning

### 3. Supervised Learning